In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# Load the document
loader = PyPDFLoader("../../lesa/samples/documents/Microprocessor.pdf")
documents = loader.load()

# Split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
docs = text_splitter.split_documents(documents=documents)

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Load embedding model
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
# model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    # model_kwargs=model_kwargs
)

# Create FAISS vector store
vectorstore = FAISS.from_documents(docs, embeddings)

# Save and reload the vector store
vectorstore.save_local("faiss_index_")
persisted_vectorstore = FAISS.load_local("faiss_index_", embeddings, allow_dangerous_deserialization=True)

# Create a retriever
retriever = persisted_vectorstore.as_retriever()

/var/folders/d8/4x7rmwkx477268kv4n9m0nm80000gn/T/ipykernel_85080/2148270060.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/Users/shantanu/Desktop/lesa/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_ollama import OllamaLLM

# Initialize the LLaMA model
llm = OllamaLLM(model="llama3.1:latest")

# Test with a sample prompt
# response = llm.invoke("Tell me a joke")
# print(response)

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
            ("system", """Answer any use questions based solely on the context below:

<context>
{context}
</context>"""),
            ("human", """{input}"""),
])

In [5]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

combine_docs_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
qa = create_retrieval_chain(retriever=persisted_vectorstore.as_retriever(), combine_docs_chain=combine_docs_chain)

# Interactive query loop
while True:
    query = input("Type your query (or type 'Exit' to quit): \n")
    if query.lower() == "exit":
        break
    result = qa.invoke({"input": query})
    print(result['answer'])

Hi! Is there something related to microprocessors you'd like to know or discuss?
